## Load result data

In [1]:
import json 

with open("experiment_results.json", "r") as results_file:
    results = json.load(results_file)

## RQ1: How does the degree of redundancy within a population influence the behavior of the proposed simulation approach?

In [2]:
# Filter out and format experiment results with the varying_redundancy tag


In [3]:
# Create a line chart with simulation time on y and redundancy level 
# on x. One line for the runs with caching enabled, one line for the 
# runs with cache size 0.


In [4]:
# Create a stack chart of the durations of different algorithm steps 
# for all runs where cache size = 100.


In [5]:
# Create box plots for sequence length and frequency of cached ngrams
# at different redundancy levels.


## RQ2.1: How does the amount of gates in the population influence simulation speed?

In [6]:
# Filter out and format experiment results with the varying_qubits tag


In [7]:
# Create a line chart with simulation time on y and qubit count 
# on x. One line for the runs with caching enabled, one line for the 
# runs with cache size 0.


## RQ2.2: How does the amount of qubits per circuit influence simulation speed?

In [8]:
# Filter out and format experiment results with the varying_gates tag


In [9]:
# Create a line chart with simulation time on y and gate count per circuit 
# on x. One line for the runs with caching enabled, one line for the 
# runs with cache size 0.


## RQ3.1: How does the amount of reordering steps influence simulation speed?

In [10]:
# Filter out and format experiment results with the reordering_varying_steps tag


In [11]:
# Create a line chart with simulation time on y and reordering step count 
# on x. 


## RQ3.2: How does the amount of reordering steps influence the lengths and frequencies of the cached gate sequences?

In [12]:
# Filter out and format experiment results with the reordering_varying_redundancy tag


In [13]:
# TBD: How to best visualize change in redundancy depending on starting redundancy

In [14]:
# Create box plots for sequence length and frequency of cached ngrams
# at different reordering step counts.
